In [41]:
import os
import fnmatch


def find_files(root_folder, extensions):
    matches = []
    for root, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if any(fnmatch.fnmatch(filename, f'*.{ext}') for ext in extensions):
                matches.append(os.path.join(root, filename))
    return matches


def get_docs():
    root_folder = '/home/dev/master-thesis/data'
    docs_extensions = ['xsd', 'pdf']
    files = find_files(root_folder, docs_extensions)
    formatted_list = "\n".join([f"- {file}" for file in files])
    return formatted_list


print(get_docs())

- /home/dev/master-thesis/data/bygning/bygningspunkt_docs.xsd
- /home/dev/master-thesis/data/flom/flomsoner_docs.xsd
- /home/dev/master-thesis/data/other/build_buff.xsd
- /home/dev/master-thesis/data/kulturminner/kulturminner_docs.xsd
- /home/dev/master-thesis/data/elveg/elveg_docs.xsd
- /home/dev/master-thesis/data/adresse/adresse_docs.xsd
- /home/dev/master-thesis/data/friluft/friluftsområder_docs.pdf
- /home/dev/master-thesis/data/AR50/ar50_docs.xsd
- /home/dev/master-thesis/data/kvikkleire/kvikkleire_docs.xsd


In [46]:
from langchain_community.document_loaders import UnstructuredXMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

load_dotenv('../../.env')

ar50_path = '/home/dev/master-thesis/data/AR50/ar50_docs.xsd'

loader = UnstructuredXMLLoader(ar50_path, mode='elements', xml_keep_tags=True)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=500,
    length_function=len,
    is_separator_regex=False
)

docs_splitted = text_splitter.split_documents(docs)

docs_splitted[9].page_content

'<annotation>\r\n          <documentation>Measured with rate of climb indicator: Spesielle metoder: Målt med stigningsmåler</documentation>\r\n        </annotation>\r\n      </enumeration>\r\n      <enumeration value="77">\r\n        <annotation>\r\n          <documentation>Punkt fastsatt ut fra et grunnlag (kart, bilde), f.eks ved partenes enighet ved en oppmålingsforretning</documentation>\r\n        </annotation>\r\n      </enumeration>\r\n      <enumeration value="78">\r\n        <annotation>\r\n          <documentation>Stipulated by court decision or royal decree: Geometri fastsatt ved dom, lov, traktat eller kongelig resolusjon</documentation>\r\n        </annotation>\r\n      </enumeration>\r\n      <enumeration value="79">\r\n        <annotation>\r\n          <documentation>????: Annet (spesifiseres i filhode)</documentation>\r\n        </annotation>\r\n      </enumeration>\r\n      <enumeration value="80">\r\n        <annotation>\r\n          <documentation>Free-hand drawing: 

In [55]:
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
from langchain_core.documents import Document


tree = ET.parse(ar50_path)
root = tree.getroot()

# Iterate through each child of the <schema> element
docs = [Document(page_content=parseString(ET.tostring(child)
                                          ).toprettyxml(newl="")) for child in root]

docs

[Document(page_content='<?xml version="1.0" ?><xs:annotation xmlns:xs="http://www.w3.org/2001/XMLSchema">\t\n    \t<xs:documentation>Denne spesifikasjonen beskriver datasettet AR50 – Arealressurskart for målestokk 1:50 000. Dette er et landsdekkende datasett som viser hovedtyper av arealressurser i målestokk 1:50 000 tilpasset bruk i målestokker fra 1:20 000 til 1:100 000. AR50 fremstilt ved en generalisering av FKB-AR5 under tregrensa og tolking av satellittbilder over tregrensa. For klassifisering av treslag har det tillegg har det vært benyttet N50 skog uten registrert treslag der det ikke finnes AR5. Figurer som er mindre enn 15 dekar vises ikke på kartet, men blir slått sammen med tilgrensende områder.</xs:documentation>\t\n  </xs:annotation>'),
 Document(page_content='<?xml version="1.0" ?><xs:import xmlns:xs="http://www.w3.org/2001/XMLSchema" namespace="http://www.opengis.net/gml/3.2" schemaLocation="http://schemas.opengis.net/gml/3.2.1/gml.xsd"/>'),
 Document(page_content='<?xm

In [56]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


faiss_index = FAISS.from_documents(docs, OpenAIEmbeddings())

for doc in faiss_index.similarity_search("forest", k=5):
    print(doc.page_content + f'\n\n{"-" * 100}\n')

<?xml version="1.0" ?><xs:simpleType xmlns:xs="http://www.w3.org/2001/XMLSchema" name="ArealressursSkogbonitetType">	
    	<xs:annotation>		
      		<xs:documentation>AreaResourceForestSiteQuality: inndeling etter arealets evne til å produsere trevirke


-- Definition - - 
categorization according to the area's ability to produce wood</xs:documentation>		
    	</xs:annotation>	
    	<xs:union memberTypes="app:ArealressursSkogbonitetEnumerationType app:ArealressursSkogbonitetOtherType"/>	
  </xs:simpleType>

----------------------------------------------------------------------------------------------------

<?xml version="1.0" ?><xs:simpleType xmlns:xs="http://www.w3.org/2001/XMLSchema" name="ArealressursSkogbonitetEnumerationType">	
    	<xs:annotation>		
      		<xs:documentation>AreaResourceForestSiteQuality: inndeling etter arealets evne til å produsere trevirke


-- Definition - - 
categorization according to the area's ability to produce wood</xs:documentation>		
    	</xs:annota

In [40]:
import os
import fnmatch
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.runnables import RunnableLambda



def find_files(root_folder, extensions):
    matches = []
    for root, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if any(fnmatch.fnmatch(filename, f'*.{ext}') for ext in extensions):
                matches.append(os.path.join(root, filename))
    return matches


def get_docs():
    root_folder = '/home/dev/master-thesis/data'
    docs_extensions = ['xsd', 'pdf']
    files = find_files(root_folder, docs_extensions)
    formatted_list = "\n".join([f"- {file}" for file in files])
    return formatted_list
    

@tool('get_full_docs', return_direct=True)
def get_full_docs_tool(path: str):
    """Tool to read the documentation located at the path."""
    try:
        with open(path, 'r') as file:    
            return file.read()
    except: 
        return f'Failed to read file at {path}'



async def get_full_documentation(dataset_name: str):
    llm = ChatOpenAI(model=os.getenv('GPT3_MODEL_NAME'))

    prompt = ChatPromptTemplate.from_messages([
        ('system', (
            "The following docs are available:\n"
            f"{get_docs()}"
        )),
        ('human', 'Give me the docs for {dataset_name}.')
    ])

    chain = (
        prompt 
        | llm.bind_tools(
            tools=[get_full_docs_tool],
            tool_choice={'type': 'function', 'function': {'name': 'get_full_docs'}}
        )
        | JsonOutputToolsParser()
        | RunnableLambda(lambda x: get_full_docs_tool(x[0]['args']))
    )

    return await chain.ainvoke({'dataset_name': dataset_name})


await get_full_documentation('veg')

'<?xml version="1.0" encoding="UTF-8"?><schema xmlns="http://www.w3.org/2001/XMLSchema" xmlns:app="http://skjema.geonorge.no/SOSI/produktspesifikasjon/Elveg/2.0" xmlns:gml="http://www.opengis.net/gml/3.2" xmlns:sc="http://www.interactive-instruments.de/ShapeChange/AppInfo" elementFormDefault="qualified" targetNamespace="http://skjema.geonorge.no/SOSI/produktspesifikasjon/Elveg/2.0" version="2.0.1">\n  <annotation>\n    <documentation>Elveg 2.0 inneholder informasjon om alle europa-, riks-, fylkes-, kommunale og skogsbilveger, samt alle private veger unntatt korte, private blindveger. Korte veger er veger med lengde mindre enn 50 m. Veger med unike adresser er med uansett lengde. Bilferjestrekninger skal være representert og gang- og sykkelveger kan være med. Vegnettet er strukturert som et nettverk av veglenker med restriksjoner i form av lineære referanser. \nVegnettet ajourføres i Nasjonal Vegdatabank, NVDB.</documentation>\n  </annotation>\n  <import namespace="http://www.interactiv

In [25]:
print(get_docs())

- /home/dev/master-thesis/data/bygning/bygningspunkt_docs.xsd
- /home/dev/master-thesis/data/flom/flomsoner_docs.xsd
- /home/dev/master-thesis/data/other/build_buff.xsd
- /home/dev/master-thesis/data/kulturminner/kulturminner_docs.xsd
- /home/dev/master-thesis/data/elveg/elveg_docs.xsd
- /home/dev/master-thesis/data/adresse/adresse_docs.xsd
- /home/dev/master-thesis/data/friluft/friluftsområder_docs.pdf
- /home/dev/master-thesis/data/AR50/ar50_docs.xsd
- /home/dev/master-thesis/data/kvikkleire/kvikkleire_docs.xsd
